In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from imblearn.under_sampling import RandomUnderSampler
from tensorboard.notebook import display

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
combined_df = pd.read_csv('clean_combined_df.csv')

In [ ]:
# Path to Dataset - Lynch
combined_df = pd.read_csv('/content/drive/MyDrive/CSE572/CSE572 Team 36 Project/clean_combined_df_binned.csv')

In [3]:
# Path to Dataset - Tejas
combined_df = pd.read_csv('/content/drive/MyDrive/CSE572 Data Mining/Initial Dataset/BRFSS Data/clean_combined_df_binned.csv')

In [ ]:
combined_df.sample(5)

,Diabetes,HighBloodPressure,HighCholesterol,Smoker,HadStroke,HadHeartDiseaseorAtack,PhysicallyActive,ConsumesFruits,ConsumesVeggies,HeavyAlcoholConsumption,GeneralHealthCondition,Gender,AgeBand,HighestLevelOfEducation,IncomeLevel,BMI_Category,MentalHealthCondition_Category,PhysicalHealthCondition_Category
179956,0,0,0,0,0,0,0,1,1,0,1,1,3,4,2,6,1,1
1389,0,1,0,0,0,0,1,1,0,0,3,0,13,5,4,3,1,1
221224,0,0,0,1,0,0,1,0,0,0,1,1,12,5,7,3,1,1
163532,0,0,0,1,0,0,1,1,1,0,2,1,9,6,6,4,1,1
69276,0,0,0,1,0,0,1,1,1,0,1,1,3,6,8,2,1,1


In [ ]:
combined_df.Diabetes.value_counts()

,count
Diabetes,
0,415502
2,70362


In [ ]:
combined_df = combined_df[combined_df.Diabetes != 1]
combined_df.replace(2.0, 1.0, inplace=True)

In [ ]:
combined_df.Diabetes.value_counts()

,count
Diabetes,
0,415502
1,70362


Start of Stratified Undersampling code

In [ ]:
# Split the data into test and train dataframes. The test split will not be undersampled, so we just immediately split it into X and y
# The train split will be undersampled.
train_df, test_df = train_test_split(combined_df, test_size = 0.2, random_state=11)
X_test = test_df.drop(columns=["Diabetes"])
y_test = test_df['Diabetes']
print(X_test.shape)
print(y_test.shape)
print(y_test.value_counts())

(97173, 17)
(97173,)
Diabetes
0    83117
1    14056
Name: count, dtype: int64


In [ ]:
# Using the train dataframe, separate the majority class (nondiabetic) from the minority class (diabetic)
train_majority = train_df[train_df["Diabetes"] == 0]
train_minority = train_df[train_df["Diabetes"] == 1]

desired_ratio = 2.0

new_majority_size = int(desired_ratio * len(train_minority))

# Use the train test split function to get a random sample of datapoints from the majority class only.
_, train_majority_undersampled = train_test_split(train_majority, test_size=new_majority_size, random_state=30)
print(train_minority.shape)
print(train_majority.shape)
print(train_majority_undersampled.shape)

(56306, 18)
(332385, 18)
(112612, 18)


In [ ]:
# Recombine the minority class data with the now undersampled majority
train_undersampled = pd.concat([train_minority, train_majority_undersampled])
train_undersampled.Diabetes.value_counts()

,count
Diabetes,
0,112612
1,56306


In [ ]:
# Create X and y for train from undersampled data
X_train = train_undersampled.drop(columns=["Diabetes"])
y_train = train_undersampled["Diabetes"]
print(X_train.shape)
print(y_train.shape)
print(y_train.value_counts())

(168918, 17)
(168918,)
Diabetes
0    112612
1     56306
Name: count, dtype: int64


End of Stratified undersampling code

Resampling first and then splitting

In [ ]:
X = combined_df.drop(columns=['Diabetes'])  # Features
y = combined_df['Diabetes']  # Target variable

In [ ]:
# Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

In [ ]:
# Resample dataset using RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train, y_train)

In [ ]:
X_train_rus.shape

(112612, 17)

In [26]:
# Tejas
# SMOTE ENN train test split
X_train = pd.read_csv('/content/drive/MyDrive/X_train_smenn.csv')
y_train = pd.read_csv('/content/drive/MyDrive/y_train_smenn.csv')
X_test = pd.read_csv('/content/drive/MyDrive/X_test_smenn.csv')
y_test = pd.read_csv('/content/drive/MyDrive/y_test_smenn.csv')
y_train.replace(2.0, 1.0, inplace=True)
y_test.replace(2.0, 1.0, inplace=True)

print(y_train.value_counts())
print(y_test.value_counts())

Diabetes
1           248159
0           175002
Name: count, dtype: int64
Diabetes
0           83003
1           14170
Name: count, dtype: int64


In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=150, random_state=11)
rf_classifier.fit(X_train_rus, y_train_rus)

In [ ]:
y_pred = rf_classifier.predict(X_test)

In [ ]:
print("Random Forest Baseline Results:\n")
print("Class count:\n", y.value_counts())
print("")
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("")
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Random Forest Baseline Results:

Class count:
 Diabetes
0    415502
1     70362
Name: count, dtype: int64

Accuracy: 0.7013985366305455

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.69      0.80     83117
           1       0.29      0.76      0.42     14056

    accuracy                           0.70     97173
   macro avg       0.62      0.72      0.61     97173
weighted avg       0.85      0.70      0.74     97173


Confusion Matrix:
 [[57507 25610]
 [ 3406 10650]]


In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

METRICS = [
      keras.metrics.Accuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]
num_experts = 8
num_selected_experts = 8

In [28]:
# Define the expert network
def create_expert():
    model = keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))  # Output probability for binary classification
    return model

In [29]:
# Define the gating network
def create_gating_network(num_experts):
    model = keras.Sequential()
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(num_experts, activation='softmax'))  # Softmax to get probabilities for experts
    return model

In [30]:
class MoELayer(layers.Layer):
    def __init__(self, num_experts, num_selected_experts=2):
        super(MoELayer, self).__init__()
        self.num_experts = num_experts # Number of expert models to use
        self.num_selected_experts = num_selected_experts # Number of experts to combine the outputs of
        self.experts = [create_expert() for _ in range(num_experts)]
        self.gate = create_gating_network(num_experts)  # The gate decides which experts to route to.

    def call(self, inputs):
        gating_output = self.gate(inputs)
        # print(f"Gating output: {gating_output.shape}")

        expert_outputs = [expert(inputs) for expert in self.experts]
        expert_outputs = tf.stack(expert_outputs, axis=1)
        # print(f"Expert predictions: {expert_outputs.shape}")

        top_k_probs, top_k_indices = tf.nn.top_k(gating_output, num_selected_experts)
        # print(f"Top K probs: {top_k_probs.shape}")
        # print(f"Top K indices: {top_k_indices.shape}")

        top_k_predictions = tf.gather(expert_outputs, top_k_indices, axis=1, batch_dims=1)
        # print(f"Top K predictions: {top_k_predictions.shape}")

        weighted_predictions = top_k_probs[..., None] * top_k_predictions
        # print(f"Weighted predictions: {weighted_predictions.shape}")

        combined_output = tf.reduce_sum(weighted_predictions, axis=1)
        # print(f"Combined predictions: {combined_output.shape}")

        return combined_output


In [31]:
class MoEModel(keras.Model):
    def __init__(self, num_experts, num_selected_experts=2):
        super(MoEModel, self).__init__()
        self.moe_layer = MoELayer(num_experts, num_selected_experts)
        self.dense = layers.Dense(1, activation="sigmoid")  # Example output layer

    def call(self, inputs):
        x = self.moe_layer(inputs)
        x = self.dense(x)
        return x

In [32]:
model = MoEModel(num_experts, num_selected_experts)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)
model.summary()

Model: "mo_e_model_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mo_e_layer_1 (MoELayer)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_55 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [33]:
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=256,
)

Epoch 1/10
1653/1653 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - accuracy: 0.0000e+00 - auc: 0.9017 - loss: 0.5060 - prc: 0.9018 - precision: 0.8838 - recall: 0.8294
Epoch 2/10
1653/1653 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.0000e+00 - auc: 0.9417 - loss: 0.3440 - prc: 0.9451 - precision: 0.8911 - recall: 0.9224
Epoch 3/10
1653/1653 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - accuracy: 0.0000e+00 - auc: 0.9516 - loss: 0.2991 - prc: 0.9588 - precision: 0.8975 - recall: 0.9196
Epoch 4/10
1653/1653 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.0000e+00 - auc: 0.9545 - loss: 0.2800 - prc: 0.9626 - precision: 0.8988 - recall: 0.9193
Epoch 5/10
1653/1653 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.0000e+00 - auc: 0.9557 - loss: 0.2707 - prc: 0.9644 - precision: 0.8987 - recall: 0.9191
Epoch 6/10
1653/1653 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.0000e+00 - auc: 0.9564 - loss: 0.2658 - prc: 0.9656 - precision: 0.8994 - recall: 0.9196
Epoch 7/10
1653/1653 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/st

In [34]:
Y_pred_moe = model.predict(X_test)
Y_pred_moe = (Y_pred_moe > 0.5).astype(float)

3037/3037 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step


In [35]:
import numpy as np

print(f"Mixture of Experts Baseline Results with {num_experts} experts and {num_selected_experts} selected experts (K):\n")
unique, counts = np.unique(Y_pred_moe, return_counts=True)
print(unique)
print(counts)
print("")
accuracy_moe = accuracy_score(y_test, Y_pred_moe)
print("Accuracy:", accuracy_moe)
print("")
print("Classification Report:\n", classification_report(y_test, Y_pred_moe))
print("")
print("Confusion Matrix:\n", confusion_matrix(y_test, Y_pred_moe))

Mixture of Experts Baseline Results with 8 experts and 8 selected experts (K):

[0. 1.]
[54695 42478]

Accuracy: 0.6638778261451226

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.63      0.76     83003
           1       0.28      0.85      0.42     14170

    accuracy                           0.66     97173
   macro avg       0.62      0.74      0.59     97173
weighted avg       0.86      0.66      0.71     97173


Confusion Matrix:
 [[52518 30485]
 [ 2177 11993]]
